<a href="https://colab.research.google.com/github/zaidsallam10/Sequence2Sequence_LSTM_RNN/blob/master/Seq2Seq_RNN_LSTM_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Data Importing
with open('ara.txt', 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')

lines[100]
# len(lines)

'Tom left.\tلقد غادر توم.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2005178 (CK) & #2005379 (Asma)'

In [3]:
# Data Splitting
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [4]:
num_samples = 10000


for line in lines[: min(num_samples, len(lines) - 1)]:
  splited=line.split('\t')
  input_text=splited[0]
  target_text=splited[1]
  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)


print(input_texts[4])
print(target_texts[4])

Stop!
	قف!



In [5]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 111
Max sequence length for inputs: 41
Max sequence length for outputs: 67


In [6]:
input_token_index=dict([(char,i) for i,char in enumerate(input_characters)])

target_token_index=dict([(char,i) for i,char in enumerate(target_characters)])

In [7]:
import numpy as np

encoder_input_data = np.zeros(
  (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
  dtype='float32')
decoder_input_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')
decoder_target_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')


print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(10000, 41, 71)
(10000, 67, 111)
(10000, 67, 111)


In [8]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.
  for t, char in enumerate(target_text):
    # decoder_target_data is ahead of decoder_input_data by one timestep
    decoder_input_data[i, t, target_token_index[char]] = 1.
    if t > 0:
      # decoder_target_data will be ahead by one timestep
      # and will not include the start character.
      decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [18]:
# --------------------------------------------------------------------------


# Modeling
import keras,tensorflow
from keras.models import Model
from keras.layers import Input,Dense,LSTM
import numpy as np

# hyperparameters
batch_size = 64  # batch size for training
epochs = 50  # number of epochs to train for
latent_dim = 256  # latent dimensionality of the encoding space



encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]



decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


model = Model(inputs=[encoder_inputs, decoder_inputs], 
              outputs=decoder_outputs)

In [19]:
# Training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()


model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)


Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None, 71)     0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, None, 111)    0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, 256), (None, 335872      input_5[0][0]                    
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(None, None, 256),  376832      input_6[0][0]                    
                                                                 lstm_3[0][1]               

In [21]:
model.save('seq2seq_eng-ger.h5')


# Save the weights
model.save_weights('model_weights_seq2seqEnToAr.h5')

# Save the model architecture
with open('model_architecture_seq2seqEnToAr.json', 'w') as f:
    f.write(model.to_json())



from google.colab import files
files.download('model_weights_seq2seqEnToAr.h5')
files.download('model_architecture_seq2seqEnToAr.json')
files.download('seq2seq_eng-ger.h5')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.load_weights('seq2seq_eng-ger.h5')

In [23]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
  decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
  [decoder_inputs] + decoder_states_inputs,
  [decoder_outputs] + decoder_states)

In [24]:
# reverse-lookup token index to turn sequences back to characters
reverse_input_char_index = dict(
  (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
  (i, char) for char, i in target_token_index.items())

In [25]:
def decode_sequence(input_seq):
  # encode the input sequence to get the internal state vectors.
  states_value = encoder_model.predict(input_seq)
  
  # generate empty target sequence of length 1 with only the start character
  target_seq = np.zeros((1, 1, num_decoder_tokens))
  target_seq[0, 0, target_token_index['\t']] = 1.
  
  # output sequence loop
  stop_condition = False
  decoded_sentence = ''
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict(
      [target_seq] + states_value)
    
    # sample a token and add the corresponding character to the 
    # decoded sequence
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]
    decoded_sentence += sampled_char
    
    # check for the exit condition: either hitting max length
    # or predicting the 'stop' character
    if (sampled_char == '\n' or 
        len(decoded_sentence) > max_decoder_seq_length):
      stop_condition = True
      
    # update the target sequence (length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.
    
    # update states
    states_value = [h, c]
    
  return decoded_sentence

In [26]:
# Make Predictions

for seq_index in range(500):
  input_seq = encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  print('-')
  print('Input sentence:', input_texts[seq_index])
  print('Decoded sentence:', decoded_sentence)

-
Input sentence: Hi.
Decoded sentence: هو يكون هنا

-
Input sentence: Run!
Decoded sentence: الرس!

-
Input sentence: Help!
Decoded sentence: الجويد يحترن.

-
Input sentence: Jump!
Decoded sentence: أعطني الكتاب.

-
Input sentence: Stop!
Decoded sentence: توقفي عن السؤاي!

-
Input sentence: Go on.
Decoded sentence: الحب وعد إلى الماز.

-
Input sentence: Go on.
Decoded sentence: الحب وعد إلى الماز.

-
Input sentence: Hello!
Decoded sentence: الجويد يحترن.

-
Input sentence: Hurry!
Decoded sentence: هل تحب التفكات؟

-
Input sentence: Hurry!
Decoded sentence: هل تحب التفكات؟

-
Input sentence: I see.
Decoded sentence: أنا أعرفه

-
Input sentence: I won!
Decoded sentence: لدي الكثير من الأختاج.

-
Input sentence: Relax.
Decoded sentence: استمع إلي

-
Input sentence: Smile.
Decoded sentence: أراك ما يمكنك.

-
Input sentence: Cheers!
Decoded sentence: الحقيقة.

-
Input sentence: Got it?
Decoded sentence: أعطني المصور.

-
Input sentence: He ran.
Decoded sentence: إنه يحبها.

-
Input sentence